# Data Preprocessing

In [1]:
import json
import pandas as pd
import osmnx as ox
import networkx as nx
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from os.path import join
from datetime import timedelta
from itertools import product
import random


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.spatial import cKDTree
from scipy.special import factorial


## Load Raw Data

In [2]:
input_path = join("io", "input")
output_path = join("io", "output")
experiments_path = join("io", "experiments")
plots_path = join(experiments_path, "plots")
traffic_data_raw_path = join(input_path, "traffic_data")
coordinates_columns = ['id', 'latitude', 'longitude']

In [3]:
with open(join(input_path, "base_bus_network.json"), 'r', encoding='utf-8') as file:
    bus_network = json.load(file)

stops_df = pd.read_csv(join(input_path, "base_stops.csv"), encoding='utf-8', sep=';')
traffic_edges_mapping_df = pd.read_csv(join(input_path, "base_network_edges_imet_map.csv"), encoding='utf-8', sep=';')

## Create Edges

In [4]:
def create_edges(bus_network, stops_df):

    # Create an empty DataFrame for edges
    edges = []

    # Parse dictionary to create edges
    for bus, stops in bus_network.items():
        for i in range(len(stops)-1):
            start_code, start_name = stops[i].split('-', 1)
            end_code, end_name = stops[i+1].split('-', 1)
            edges.append({
                'start_node_name': start_name,
                'end_node_name': end_name,
                'start_node_id': start_code,
                'end_node_id': end_code,
                'edge_id': f'{start_code}_{end_code}',
                'bus': [bus]
            })
    
    # Convert list to DataFrame
    edges_df = pd.DataFrame(edges)
    
    edges_df['start_node_id'] = edges_df['start_node_id'].astype(str)
    edges_df['end_node_id'] = edges_df['end_node_id'].astype(str)
    
    stops_df['id'] = stops_df['id'].astype(str)
    
    # If an edge appears in multiple buses, concatenate bus names
    edges_df = edges_df.groupby(['start_node_id', 'end_node_id', 'start_node_name', 'end_node_name', 'edge_id']).agg({
        'bus': lambda x: list(set([b for sublist in x for b in sublist]))}).reset_index()
    
    edges_df = edges_df.merge(stops_df, how='left', left_on='start_node_id', right_on='id')
    edges_df.rename(columns={'latitude': 'start_lat', 'longitude': 'start_long'}, inplace=True)
    edges_df.drop(columns='id', inplace=True)
    
    edges_df = edges_df.merge(stops_df, how='left', left_on='end_node_id', right_on='id')
    edges_df.rename(columns={'latitude': 'end_lat', 'longitude': 'end_long'}, inplace=True)
    edges_df.drop(columns='id', inplace=True)
    edges_mapped = pd.merge(edges_df, traffic_edges_mapping_df[['edge_id', 'edge_id_traffic']], on='edge_id', how='inner')
    
    return edges_mapped
    

In [5]:
# Function to find the nearest nodes on the graph
def get_nearest_node(lat, lon, graph):
    return ox.distance.nearest_nodes(graph, lon, lat)

In [6]:
def calculate_distances(edges_df):
    # Calculate the driving distances for each pair of start and end nodes in edges_df
    
    # Define the place and download the road network graph for driving
    place_name = "Thessaloniki, Greece"
    G = ox.graph_from_place(place_name, network_type='drive')

    driving_distances = []  # List to store the distances
    for index, row in edges_df.iterrows():
        start_lat = row['start_lat']
        start_lon = row['start_long']
        end_lat = row['end_lat']
        end_lon = row['end_long']

        # Get the nearest network nodes to these coordinates
        start_node = get_nearest_node(start_lat, start_lon, G)
        end_node = get_nearest_node(end_lat, end_lon, G)

        # Compute the shortest path based on the length of the edges
        route = nx.shortest_path(G, start_node, end_node, weight='length')

        # Calculate the total distance of the route in meters
        total_distance = 0
        for i in range(len(route) - 1):
            edge_data = G.get_edge_data(route[i], route[i + 1])[0]  # Assumes there's at least one edge between nodes
            total_distance += edge_data['length']

        driving_distances.append(total_distance)

    # Assign the computed distances to a new column in the DataFrame
    edges_df['driving_distance'] = driving_distances

    return edges_df

## Preprocess Traffic Data

In [7]:
def process_csv(file_path, valid_ids, chunksize=1000):  # Adjust chunksize based on your needs
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        filtered_chunk = chunk[chunk['Link_ID'].isin(valid_ids)]
        chunks.append(filtered_chunk)
    return pd.concat(chunks)


In [8]:
def merge_traffic_data(edges_df, traffic_data_raw_path, output_path, output_name = "interm_traffic_data_all_merged.csv"):
    # Gather all valid 'edge_id_traffic' as a filter
    valid_ids = set(edges_df['edge_id_traffic'])

    output_file_path = join(output_path, output_name)
    
    if os.path.exists(output_file_path):
        os.remove(output_file_path)
    
    # Directory containing all CSV files
    header_written = False  # To ensure the header is written only once
    
    # Process each CSV file
    for file_name in tqdm(os.listdir(traffic_data_raw_path)):
        if file_name.endswith('.csv'):
            print(file_name)
            file_path = os.path.join(traffic_data_raw_path, file_name)
            filtered_df = process_csv(file_path, valid_ids)
    
            if not header_written:
                filtered_df.to_csv(output_file_path, mode='w', index=False)  # Write new file with header
                header_written = True
            else:
                filtered_df.to_csv(output_file_path, mode='a', index=False, header=False)
    
    return header_written


In [9]:
def refactor_traffic_data(edges_df, traffic_data_all_merged, subset, subset_start_date):
    edges_columns = ['start_node_id', 'end_node_id', 'edge_id', 'edge_id_traffic', 'driving_distance']
    traffic_data_columns = ['Link_ID', 'Timestamp', 'Avg_speed']
    final_columns = ['node_id', 'timestamp', 'ETA']
    
    edges_df = edges_df[edges_columns]
    traffic_data_all_merged = traffic_data_all_merged[traffic_data_columns].copy()
    traffic_data_all_merged.rename(columns={'Link_ID': 'edge_id_traffic', 'Timestamp': 'timestamp', 'Avg_speed': 'avg_speed'}, inplace=True)
    
    
    traffic_data_all_merged = pd.merge(edges_df, traffic_data_all_merged, on='edge_id_traffic', how='inner')
    traffic_data_all_merged.rename(columns={'edge_id': 'node_id'}, inplace=True)

    traffic_data_all_merged['ETA'] = traffic_data_all_merged['avg_speed']
    traffic_data_all_merged = traffic_data_all_merged[final_columns]
    
    
    traffic_data_all_merged['timestamp'] = traffic_data_all_merged['timestamp'].apply(
        lambda x: x + " 00:00:00" if " " not in x else x
    )
    traffic_data_all_merged['timestamp'] = pd.to_datetime(traffic_data_all_merged['timestamp'])
    
    if subset:
        traffic_data_all_merged = traffic_data_all_merged[traffic_data_all_merged['timestamp'] > subset_start_date]
    
    traffic_data_all_merged.to_csv(join(output_path, "stg_traffic_data_all_merged.csv"), sep=',', encoding='utf-8', index=False)

    return traffic_data_all_merged


In [10]:
def fulfill_missing_targets(df):
    # Extract the hour from the timestamp
    df['hour'] = df['timestamp'].dt.hour
    
    # Calculate the mean 'ETA' for each 'node_id' for each hour across all dates
    hourly_means = df.groupby(['node_id', 'hour'])['ETA'].mean().reset_index()
    hourly_means.rename(columns={'ETA': 'hourly_ETA'}, inplace=True)
    
    # Generate all node-timestamp combinations
    all_node_ids = df['node_id'].unique()
    all_hours = df['hour'].unique()
    
    # Add full timestamp range to all_combinations for complete timeseries
    min_date = df['timestamp'].min().normalize()  # Normalize to remove time part
    max_date = df['timestamp'].max().normalize()
    all_times = pd.date_range(start=min_date, end=max_date + timedelta(days=1), freq='15min')
    all_times = all_times[(all_times.hour >= 7) & (all_times.hour <= 23)]
    
    
    all_combinations = pd.DataFrame(list(product(all_node_ids, all_times)), columns=['node_id', 'timestamp'])
    all_combinations['hour'] = all_combinations['timestamp'].dt.hour
    
    # Merge the full combinations with the original data
    full_df = pd.merge(all_combinations, df.drop(columns='hour'), on=['node_id', 'timestamp'], how='left')
    missing_data = full_df[full_df['ETA'].isnull()][['node_id', 'timestamp']]

    full_df.sort_values(by=['node_id', 'timestamp'], inplace=True)
    full_df['ETA'] = full_df.groupby('node_id')['ETA'].ffill().bfill()
    
    # Merge hourly mean speeds into the full DataFrame
    full_df = pd.merge(full_df, hourly_means, on=['node_id', 'hour'], how='left')
    
    # Fill in missing 'ETA' values using the hourly mean 'ETA'
    full_df['ETA'] = full_df['ETA'].fillna(full_df['hourly_ETA'])
    full_df.drop(columns=['hourly_ETA'], inplace=True)

    #full_df.to_csv(join(output_path, "fct_traffic_data_all_merged.csv"), sep=',', encoding='utf-8', index=False)
    #missing_data.to_csv(join(output_path, "base_missing_raw_targets.csv"), sep=',', encoding='utf-8', index=False)
    
    return full_df

## Execution

In [13]:
subset_start_date = '2024-10-31'
subset = False

## Intermidiate Preprocessing

In [31]:
edges_df = create_edges(bus_network, stops_df[coordinates_columns].copy())
edges_df = calculate_distances(edges_df)
edges_df.to_csv(join(output_path, "interm_network_edges.csv"), sep=',', encoding='utf-8', index=False)
status =  merge_traffic_data(edges_df, traffic_data_raw_path, output_path)

## Data Cleaning & Missing Values Handling

In [35]:
edges_df = pd.read_csv(join(output_path, "interm_network_edges.csv"), encoding='utf-8', sep=',')
traffic_data_all_merged = pd.read_csv(join(output_path, "interm_traffic_data_all_merged.csv"), encoding='utf-8', sep=',')
traffic_data_all_merged = traffic_data_all_merged.drop_duplicates()
traffic_data_all_merged.to_csv(join(output_path, "interm_traffic_data_all_merged.csv"), sep=',', encoding='utf-8', index=False)

final_traffic_data_all_merged = refactor_traffic_data(edges_df, traffic_data_all_merged, subset, subset_start_date)
final_traffic_data_all_merged = fulfill_missing_targets(final_traffic_data_all_merged)
final_traffic_data_all_merged

,node_id,timestamp,hour,ETA
0,1004_1256,2022-01-01 07:00:00,7,41.0
1,1004_1256,2022-01-01 07:15:00,7,36.0
2,1004_1256,2022-01-01 07:30:00,7,45.0
3,1004_1256,2022-01-01 07:45:00,7,56.0
4,1004_1256,2022-01-01 08:00:00,8,56.0
...,...,...,...,...
16937095,45129_45113,2025-01-11 22:45:00,22,45.0
16937096,45129_45113,2025-01-11 23:00:00,23,45.0
16937097,45129_45113,2025-01-11 23:15:00,23,45.0
16937098,45129_45113,2025-01-11 23:30:00,23,45.0
